# Project 3
## Joshua Anderson

## General Aspects
1. There will be 10 periods. Players begin with 70 health.
2. Each period, the player harvests some amount of money (this amount is detailed below)
3. After harvesting, the player's health degenerates (this amount is detailed below)
4. After health degeneration, the player must spend money on Health Investments and Life Investments. Money spent on Health Investments increases health, while money spent on Life 5. 5. Investments gives the player Life Enjoyment. Any money not spent carries over into the next period.
6. A player dies if their health ever goes below 0. If a player dies, they receive 0 Life Enjoyment for the remaining periods.
7. The goal is to maximize total Life Enjoyment across all periods.

## Functions
 

Harvesting: 
1. The player earns income by harvesting black dots in a region designated by M x N cells. In our parameters (M = N = 100)
2. The player can select any set of contiguous W columns to harvest. In our parameters (W = 10)
3. When fully healthy, there are T black dots dispersed across M x W cells each period. Each black dot is worth v. In our parameters T = 100 and v = 1
4. The number of rows that can be harvested each period are given by: HarvestRows(H) = M $(1 - \gamma \frac{100 - H}{100})$ where $(\gamma = 1)$
5. The number of rows are reduced by disabling the rows in the upper and lower regions of the selected columns.
6. With these parameters, if health is 50 at the start of the period, the player has only 500 cells in which they can harvest black dots. With 80 health, they have 800 cells.
 

Degeneration:

Each period, the player loses (10 + CurrentPeriod) in health. i.e. 11 health the first period, 12 the second period, up to 20 in the last period.

Health Regeneration:

The equation for the amount of health regained given a certain Health Investment, I, and health after harvesting, H, is given by: 
$$ HealthRegained(I, H) = 100 \left( \frac{e^{k \cdot I}}{e^{k \cdot I} + \frac{100-H}{H}}\right)  - H $$

where (k = 0.01021).  Health cannot exceed 100, and is always rounded down to the nearest integer. 

Life Enjoyment:

The equation for the amount of Life Enjoyment given a certain Life Investment, L, is given by: 

$$ LifeEnjoyment(L, CurrentHealth) = c \left( \frac{CurrentHealth}{100} \right) \left( \frac{L}{L + \alpha} \right) $$

where (c = 464.53) and ($\alpha$ = 32).

CurrentHealth is the health the player has during the investment phase **INCLUDING** the amount regained this period through investments in health.

In [3]:
import numpy as np
import pandas as pd
import math
import random

# Constants
periods = 10
population_size = 50

starting_health = 70
min_health = 0

starting_enjoyment = 0

harvest_region_M = 100
harvest_region_N = 100
harvest_region_W = 10

mutate_prob = 0.05
num_children = int(population_size * .75)


In [8]:
# player class
class Player:
    def __init__(self, health, enjoyment):
        self.health = health
        self.enjoyment = enjoyment
        self.money = 0
        self.fitness = 0
        self.last_strategy = ()
    
    def select_column(self, N, W):
        self.col_index = random.randint(0, N - W)

    def get_dot_count(self):
        return sum(self.dots.sum(axis=1))
    
    def get_dots(self, W, disable_lower, disable_upper, data):
        start_idx = self.col_index
        end_idx = self.col_index + W
        self.dots = data.iloc[disable_lower:disable_upper,start_idx:end_idx]
        self.money += self.get_dot_count()

    def set_I(self, new_I):
        self.I = new_I
        self.money -= self.I

    def set_L(self, new_L):
        self.L = new_L
        self.money -= self.L

    def get_invest_props(self):
        invest_sum = self.I + self.L

        if invest_sum == 0 and self.money == 0:
            return 0, 0
        
        I_prop = self.I / (self.money + invest_sum)
        L_prop = self.L / (self.money + invest_sum)

        return I_prop, L_prop

    def reset_investments(self, refill_money=True):
        if refill_money:
            self.money += self.I + self.L

        self.I = 0
        self.L = 0

    def invest(self):
        self.enjoyment += e
        self.health += r
    
    def set_strategy(self, I_prop, L_prop):
        self.last_strategy = (round(I_prop,2), round(L_prop,2))

In [5]:
### simulation functions ###

# Harvesting
# @params:
# M x N cells to a harvesting region
# W - contiguous columns to harvest
# T - black dots in harvesting region
# v - value of the black dots in a harvesting region
def harvest(players, M = 100, N = 100, W = 10, T = 100, v = 1, gamma = 1):
    region = pd.DataFrame()
    for i in range(N):
        value = T // W
        column = np.array([v]*value + [0]*(M-value))
        np.random.shuffle(column)
        region[i] = column

    for player in players:
        # harvest rows
        if player.health > min_health:
            reduced_gamma = gamma * ((100 - player.health) / 100)
            num_rows = M * (1 - reduced_gamma)
            disable_lower = int((M - num_rows) / 2)
            disable_upper = M - disable_lower
        
            player.select_column(N, W)
            player.get_dots(W, disable_lower, disable_upper, region)

# Degeneragtion
# @params:
# curr_period - the current period of the experiment
# P - penalty added to the current period against health
def degenerate(curr_period, P = 10):
    return P + curr_period

# Health Regeneration
# @params:
# k - regen constant
# max_health - the value that health cannot exceed
def regenerate(player, k = 0.01021, max_health = 100):
    e_kI = math.exp(k*player.I)
    H_frac = (max_health - player.health) / player.health
    
    health_regen = math.floor(max_health * (e_kI / (e_kI + H_frac)) - player.health)

    if health_regen < 0:
        return 0
    elif health_regen > max_health:
        return max_health
    else:
        return health_regen

# Life Enjoyment - Fitness Function
# @params:
# c - constant multiplied by health
# alpha - constant that divides investment
def enjoyment(player, c = 464.53, alpha = 32):
    return c * (player.health / 100) * (player.L / (player.L + alpha))

In [6]:
### GA functions ###

# Generate Population
# @params:
# N - number of columns to choose from
# W - width of each column
# size - number of players in the population
def get_players(N, W, size):
    players = []
    for i in range(size):
        selection = random.randint(0, N - W)
        players.append(Player(starting_health, starting_enjoyment))
    return players

# Mutate
# @params:
# player - a given player in the experiment
# prob - the probability of a mutation
def mutate_player(player, prob):
    if random.random() < prob:
        # track current invesments
        I_prop, L_prop = player.get_invest_props()
        player.reset_investments()

        # randomly choose to increase or decrease health investment w/ 50% chance
        if random.random() < 0.5:
            mutation_prop = random.uniform(0, L_prop)
            I_prop += mutation_prop
            L_prop -= mutation_prop
        else:
            mutation_prop = random.uniform(0, I_prop)
            L_prop += mutation_prop
            I_prop -= mutation_prop

        player.set_strategy(I_prop, L_prop)
        I = I_prop * player.money
        L = L_prop * player.money

        player.set_I(round(I))
        player.set_L(round(L))

    return player

# Fitness function
# @params:
# player - a given player in the experiment
def investment_fitness(player, curr_period):
    score = 0

    score += (0.7 * player.I) + (0.3 * player.L) * player.money

    player.fitness = score


# Generate children
# @params
# players - the population of players
# num_children - the number of children to be generated
# mutate_prop - probability of mutation
def generate_children(players, num_children, mutate_prob):
    children = []

    for _ in range(num_children // 2):
        parents = random.sample(players, 2)
        choice = random.randint(0, 1)

        child = parents[choice]
        I_prop_A, L_prop_A = parents[0].get_invest_props()
        I_prop_B, L_prop_B = parents[1].get_invest_props()
        child.reset_investments()

        if choice == 0:
            while I_prop_A + L_prop_B > 1:
                L_prop_B -= random.uniform(0, L_prop_B)

            child.set_I(round(I_prop_A * child.money))
            child.set_L(round(L_prop_B * child.money))
            child.set_strategy(I_prop_A, L_prop_B)
            children.append(mutate_player(child, mutate_prob))
        
        else:
            while I_prop_B + L_prop_A > 1:
                I_prop_B -= random.uniform(0, I_prop_B)

            child.set_I(round(I_prop_B * child.money))
            child.set_L(round(L_prop_A * child.money))
            child.set_strategy(I_prop_B, L_prop_A)
            children.append(mutate_player(child, mutate_prob))

    return children

# Iterate Population
# @params:
# players - the population of players
# size - size of the output population
def tournament_survival(players, size, curr_period):
    new_players = []
    for i in range(size):
        player_1 = players[random.randint(0, len(players)-1)]
        player_2 = players[random.randint(0, len(players)-1)]

        # update health and enjoyment values
        if player_1.health > min_health:
            player_1.invest()
            investment_fitness(player_1, curr_period)
            player_1.reset_investments(refill_money=False)

        if player_2.health > min_health:
            player_2.invest()
            investment_fitness(player_2, curr_period)
            player_2.reset_investments(refill_money=False)
            
        # choose player with greater fitness
        if player_1.fitness >= player_2.fitness:
            new_players.append(player_1)
        else:
            new_players.append(player_2)
    return new_players

# Iterate GA
# @params:
# players - the population of players
# num_children - the number of children to be generated
# mutate_prob - probability of mutation
def ga_iteration(players, num_children, mutate_prob, curr_period):
    harvest(players)
    print(f"Curr_Period: {curr_period}")
    print([player.money for player in players])
    for player in players:
        if player.health > min_health:
            player.health = player.health - degenerate(curr_period)

            if player.health > min_health:
                player.set_I(random.randint(0, int(player.money)))
                player.set_L(random.randint(0, int(player.money)))
            else:
                player.health = 0
        else:
            player.health = 0
        # print_player(player)
        # print()

    children = generate_children(players, num_children, mutate_prob)
    players = tournament_survival(players + children, population_size, curr_period)

# Print player values to console
# @params:
# player - a given player in the experiment
def print_player(player):
    print(f"Health: {player.health}")
    print(f"Life Enjoyment: {player.enjoyment}")

    # if first iteration, player will not have previous investemnt
    try:
        print(f"Next Health Investment: {player.I}")
        print(f"Next Life Enjoyment Investment: {player.L}")
        print(f"Fitness: {player.fitness}")
    except:
        pass

In [7]:
### simulation ###
players = get_players(harvest_region_N, harvest_region_W, population_size)

for i in range(periods):
    ga_iteration(players, num_children, mutate_prob, i)
    # print()

### find best player ###
print("--- Best Life Enjoyment Result ---")
print([(player.last_strategy, player.fitness) for player in players])
max_player = players[0]
for player in players:
    if player.enjoyment > max_player.enjoyment:
        max_player = player
print_player(max_player)


Curr_Period: 0
[64, 75, 71, 72, 71, 66, 69, 68, 67, 71, 68, 70, 74, 69, 68, 75, 68, 77, 72, 75, 74, 66, 63, 70, 72, 69, 72, 72, 75, 75, 72, 72, 74, 65, 72, 71, 76, 69, 73, 69, 77, 68, 74, 71, 64, 75, 67, 69, 75, 64]
169.94999999999996 0
100.33847999999998 11
176.2010344827586 0
61.181999999999995 13
139.35899999999998 1
8.446 16
37.66459459459459 7
116.55479999999997 11
88.95255319148936 2
16.395176470588233 13
0.0 0
16.395176470588233 2
0.0 2
84.82721739130434 7
23.890114285714283 2
8.446 5
0.0 0
66.36142857142856 12
0.0 0
0.0 0
0.0 10
139.35899999999998 0
122.24473684210524 8
61.181999999999995 10
0.0 0
0.0 0
44.00810526315789 11
100.33847999999998 6
0.0 12
44.00810526315789 9
50.02630769230768 12
0.0 0
0.0 0
0.0 0
0.0 0
0.0 10
0.0 14
0.0 0
0.0 0
0.0 14
0.0 15
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 9
0.0 0
0.0 0
110.43543396226414 7
0.0 0
151.30405714285712 7
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
0.0 0
134.86354838709676 2
0.0 0
55.743599999999994 3
71.29995348